In [124]:
from bs4 import BeautifulSoup
from pathlib import Path
from src.preprocess import fix_text
import pandas as pd
import spacy
import re
import xml.etree.ElementTree as ET

In [25]:
nlp = spacy.load("fr_core_news_sm")

In [ ]:
def tokenize_xml(xml_code):
    tokens = []
    root = ET.fromstring(xml_code)
    _tokenize_element(root, tokens)
    return tokens

def _tokenize_element(element, tokens):
    tag = "<" + element.tag + (" " + ' '.join(f'{attr}="{value}"' for attr, value in element.attrib.items()) if len(element.attrib)>0 else '') + ">"
    tokens.append(tag)  # Start tag
    if element.text and element.text.strip():
        tokens.extend(el.text for el in nlp(element.text.strip())) # String value
    for child in element:
        if child.tag == ET.Comment:
            pass  # Comment
        else:
            _tokenize_element(child, tokens)  # Recursive call for child elements
    tokens.append("</" + element.tag + ">")  # End tag

In [131]:
PATH = Path('.data/text_tests/')
# PATH = Path('D:/Users/subje/Documents/GitHub/jusTAL/.data/text_termines')

In [132]:
texts = []
for path in PATH.glob('*.xml'):
    for disc in BeautifulSoup(fix_text(path.read_text(encoding='utf-8')), features='xml').findAll(attrs={'type':'discussion'}):
        texts.append((disc, path.stem))
    # texts.append((BeautifulSoup(path.read_text(encoding='utf-8'), features='xml').find('body').text, path.stem))
df = pd.DataFrame(texts, columns=['text', 'id'])

In [187]:
tab = []
len_words = -1
for m, text in enumerate(df.text[:2]):
    tokens = tokenize_xml(str(text))
    i=0
    # print(f'#Text={" ".join(tokens)}')
    tab.append(f'\r\r#Text={" ".join(tokens)}')
    cur_speaker = '_'
    for n, token in enumerate(tokens):
        # print(f'{m+1}-{n+1}\t{i}-{i+len(token)}\t{token}')
        speaker = target = value = relation = spk_trgt = '_'
        if re.search('<u who=', token):
            # print(re.findall('who="#(.*?)"', token))
            speaker = True
            value = 'PERSON'
            cur_speaker = f'{m+1}-{n+1}'
        if re.search('<span ana', token):
            # print(re.findall('ana="#(.*?)"', token))
            target = True
            value = 'PERSON'
            relation = 'NULL'
            spk_trgt = cur_speaker
        tab.append(f'\r{m+1}-{n+1}\t{len_words+1}-{len_words+len(token)+1}\t{token}\t{str(speaker)}\t{str(target)}\t{value}\t{relation}\t{spk_trgt}')
        len_words+=len(token)+1

In [188]:
with open('D:/Users/subje/Desktop/dddddd.tsv', 'w') as f:
    f.write('#FORMAT=WebAnno TSV 3.3\r')
    f.write('#T_SP=webanno.custom.JusTALNER|isSpeaker|isTarget|value_ner\r')
    f.write('#T_RL=webanno.custom.JusTALREL|value_rel|BT_webanno.custom.JusTALNER\r')
    for line in tab:
            f.write(line)